#

In [1]:


import argparse
# !pip install opencv-python
import cv2
import json
import numpy as np
import os
# !pip install pandas
import pandas as pd
# !pip install scipy
import scipy.misc
import shutil
import time
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parallel
import torch.optim as optim
import torchvision
import torchvision.models as models
import sys
    # caution: path[0] is reserved for script path (or '' in REPL)
# sys.path.insert(1, 'Deep-Leafsnap-master')

# !pip install scikit-image
# !pip install scikit-learn 
import utils

from scipy.stats import norm

from sklearn.neighbors import KNeighborsClassifier
# !pip install imblearn
import imblearn
from collections import Counter

from PIL import Image
from averagemeter import *
from models import *
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from torch.autograd import Variable
from torch.utils.data import sampler
from torchvision import datasets
from torchvision import transforms

import warnings

INPUT_SIZE = 224
NUM_CLASSES = 185
NUM_EPOCHS = 30*3
LEARNING_RATE = 1e-2
USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)
best_prec1 = 0
classes = []


True


In [12]:

# This file provides a long_running decorator to indicate that a function needs a long amount of time to complete and
# the computer should not enter standby. This file currently only works on Windows and is a no-op on other platforms.

#copied from Kevin Barnes/kbarnes3: https://gist.github.com/kbarnes3/3fb7d353e9bdd3efccd5

import ctypes
import platform

ES_CONTINUOUS = 0x80000000
ES_SYSTEM_REQUIRED = 0x00000001


def _set_thread_execution(state):
    ctypes.windll.kernel32.SetThreadExecutionState(state)


def prevent_standby():
    if platform.system() == 'Windows':
        _set_thread_execution(ES_CONTINUOUS | ES_SYSTEM_REQUIRED)


def allow_standby():
    if platform.system() == 'Windows':
        _set_thread_execution(ES_CONTINUOUS)


def long_running(func):
    def inner(*args, **kwargs):
        prevent_standby()
        result = func(*args, **kwargs)
        allow_standby()
        return result
    return inner

In [3]:
# model = torchvision.models.vgg16(pretrained=True)
# model = torchvision.models.vgg16(pretrained=False)

In [4]:
# model.eval()

model = torchvision.models.vgg19(pretrained=True)
# for param in model.parameters():
#     param.requires_grad = True
model.classifier[-1]=nn.Linear(4096, 185)


C:\Users\vjosv\anaconda3\envs\cuda_env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\vjosv\anaconda3\envs\cuda_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:



# model = models.resnet101(weights=False)
# model.fc = nn.Linear(2048, 185)
# model.name='resnet101'

# model = VGG('VGG19')
model.name='vgg19_pretrained'
if USE_CUDA:
    model.cuda()
# model = VGG('VGG16')
# model.name='vgg16_new'
# model = resnet101()
# model = densenet121()
# model.name='densenet121'



traindir = os.path.join('dataset', 'train')
testdir = os.path.join('dataset', 'test')
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
data_train = datasets.ImageFolder(traindir, transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize]))
data_test = datasets.ImageFolder(testdir, transforms.Compose([
    transforms.ToTensor(),
    normalize]))
classes = data_train.classes

train_loader = torch.utils.data.DataLoader(data_train, batch_size=64, shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(data_test, batch_size=64, shuffle=False, num_workers=2)



optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                      momentum=0.9, weight_decay=1e-4, nesterov=True)
criterion = nn.CrossEntropyLoss()


overload_training = False

if os.path.isfile(f'{model.name}_checkpoint.pth.tar') and not overload_training:
    checkpoint = torch.load(f'saved_models/{model.name}_checkpoint.pth.tar')
    # checkpoint = torch.load(f'saved_models/{model.name}_model_best.pth.tar')
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])

In [6]:
# # model.eval()
if not os.path.isdir(f'saved_output/{model.name}_validating_output'):
    os.mkdir(f'saved_output/{model.name}_validating_output')
    
#     (f'{model.name}_training_output')

In [7]:
train_loader = torch.utils.data.DataLoader(data_train, batch_size=64, shuffle=True, num_workers=2)

test_loader = torch.utils.data.DataLoader(data_test, batch_size=64, shuffle=False, num_workers=2)

In [8]:
def save_output_target(model,output,target,epoch,batch):
        torch.save(output,f'saved_output/{model.name}_training_output/output_e{epoch}_b{batch}')
        torch.save(target,f'saved_output/{model.name}_training_output/target_e{epoch}_b{batch}')
    
def save_output_target_validate(model,output,target,epoch,batch):
        torch.save(output,f'saved_output/{model.name}_validating_output/output_e{epoch}_b{batch}')
        torch.save(target,f'saved_output/{model.name}_validating_output/target_b{batch}')
        

def train(train_loader, model, criterion, optimizer, epoch):
    if epoch == 9:
        for param in model.parameters():
            param.requires_grad = True
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):
        # measure data loading time
        if USE_CUDA:
            input = input.cuda(non_blocking=True)
            target = target.cuda(non_blocking=True)

        data_time.update(time.time() - end)

        input_var = torch.autograd.Variable(input)
        target_var = torch.autograd.Variable(target)

        # compute output
        output = model(input_var)
        # print(output)

        loss = criterion(output, target_var)
        # print(loss)
        # if i >10:
        #     break
        
        # save_output_target(model,output,target_var,epoch,i)
        # measure accuracy and record loss
        prec1, prec5,conf = accuracy(output.data, target, topk=(1, 5))
        losses.update(loss.data.item(), input.size(0))
        top1.update(prec1.item(), input.size(0))
        top5.update(prec5.item(), input.size(0))
        
        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 100 == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  '\Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                      epoch, i, len(train_loader), batch_time=batch_time,
                      data_time=data_time, loss=losses, top1=top1, top5=top5))


def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    if (epoch-1) < 0:
        former_lr=LEARNING_RATE
    else:
        former_lr = LEARNING_RATE * (0.1 ** ((epoch-1) // 30))
    lr = LEARNING_RATE * (0.1 ** (epoch // 30))
    if (lr <= 0.0001):
        lr = 0.0001
    print('\n[Learning Rate] {:0.6f}'.format(lr))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr != former_lr

# def accuracy(output, target, topk=(1,)):
#     """Computes the precision@k for the specified values of k"""
#     maxk = max(topk)
    
#     batch_size = target.size(0)

#     _, pred = output.topk(maxk, 1, True, True)
#     pred = pred.t()

#     correct = pred.eq(target.view(1, -1).expand_as(pred))
    
#     res = []
#     for k in topk:
#     	correct_k = torch.sum(torch.reshape(correct[:k],(-1,) ).float()) #.double().sum(0)
#     	res.append(correct_k.mul_(100.0 / batch_size))
#     return res


def save_checkpoint(state, is_best, filename=f'saved_models/{model.name}_checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        print('\n[INFO] Saved Model to model_best.pth.tar')
        shutil.copyfile(filename, f'saved_models/{model.name}_model_best.pth.tar')



In [9]:

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred_t = pred.t()

    correct = pred_t.eq(target.view(1, -1).expand_as(pred_t))

    nb_classes = 185

    confusion_matrix = torch.zeros(nb_classes, nb_classes)
    _, preds = torch.max(output, 1)
    
    for t, p in zip(target.view(-1), preds.view(-1)):
        confusion_matrix[t.long(), p.long()] += 1

    # print(confusion_matrix)
    
    
    # print(confusion_matrix.diag()/confusion_matrix.sum(1))
        
    res = []
    for k in topk:
        correct_k = torch.sum(torch.reshape(correct[:k],(-1,) ).float()) #.double().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    res.append(confusion_matrix)
    return res


tt=''
within_square = ''
sample_pos =''
output_aug_kde001 = ''
weight_tensor_kde001 = ''
zero_tensor_kde001=''
def validate(val_loader, model, criterion,epoch,save_output=False):
    global output_aug_kde001, sample_pos, within_square, weight_tensor_kde001,zero_tensor_kde001,tt
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # losses_aug =  AverageMeter()
    # top1_aug = AverageMeter()
    # top5_aug = AverageMeter()
    # conf_aug = AverageMeter()
    
    # top1_aug500 = AverageMeter()
    # top5_aug500 = AverageMeter()
    # conf_aug500 = AverageMeter()
    
    # top1_aug_norm1_250sd = AverageMeter()
    # top5_aug_norm1_250sd = AverageMeter()
    # conf_aug_norm1_250sd = AverageMeter()
    
    # top1_aug_norm2_500sd = AverageMeter()
    # top5_aug_norm2_500sd = AverageMeter()
    # conf_aug_norm2_500sd = AverageMeter()
    
    # top1_aug_kde05 = AverageMeter()
    # top5_aug_kde05 = AverageMeter()
    # conf_aug_kde05 = AverageMeter()
    
    # top1_aug_kde1 = AverageMeter()
    # top5_aug_kde1 = AverageMeter()
    # conf_aug_kde1 = AverageMeter()
    
    conf = AverageMeter()
    class_correct = list(0. for i in range(185))
    class_total = list(0. for i in range(185))

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (input, target) in enumerate(val_loader):
        if USE_CUDA:
            input = input.cuda(non_blocking=True)
            target = target.cuda(non_blocking=True)
        input_var = torch.autograd.Variable(input, volatile=True)
        target_var = torch.autograd.Variable(target, volatile=True)

        # compute output
        output = model(input_var)
        
        # weight_tensor_1000 = np.ones((output.size(0),185))*0.1
        # weight_tensor_500 = np.ones((output.size(0),185))*0.1
        # weight_tensor_1500 = np.ones((output.size(0),185))*0.1


        
        # weight_tensor_normal250sd = np.ones((output.size(0),185))*stats_distance1.pdf(1000)
        # weight_tensor_normal500sd = np.ones((output.size(0),185))*stats_distance2.pdf(1000)
        # sample_pos_list = []
        # for j in range(output.size(0)):
        #     sample_pos = sample_plant_position(int(target[j]),dat)
        #     sample_pos_list.append(sample_pos)
        
        # for sample_pos in sample_pos_list:
        
            # points_in_area_list = get_points_within(sample_pos)
            # points_in_area = dat[dat['Id'].isin(points_in_area_list)]

            # points_in_area_list_500 = get_points_within(sample_pos,distance=500)
            # points_in_area_500 = dat[dat['Id'].isin(points_in_area_list_500)]
            
            
            # indexes_in_area = [ names_mapping[i] for i in list(points_in_area['Vitenskapelig navn'].unique())]


            
            # for k in indexes_in_area:
            #     weight_tensor_1000[j][k]=1

            # indexes_in_area_500 = [ names_mapping[i] for i in list(points_in_area_500['Vitenskapelig navn'].unique())]

            # for k in indexes_in_area_500:
            #     weight_tensor_500[j][k]=1

            
            # id = int(sample_pos['Id'].iloc[0])
            
            # # print(sample_pos)
            # # v = point_tree.query_ball_point([[[int(dat['Østkoordinat'].iloc[0])],int(dat['Nordkoordinat'].iloc[0])]], distance,return_sorted= True)[0]
            # v = point_tree.query_ball_point([[int(sample_pos['Østkoordinat'].iloc[0]),int(sample_pos['Nordkoordinat'].iloc[0])]], 1000)[0]
 
            
            # return_dat = dat.iloc[v]

            # return_dat = return_dat[return_dat.Id != sample_pos['Id'].iloc[0]]
            
            # points_in_area_list = list(return_dat['points'])
            # # points_in_area_list.remove(id)
        
            
            # # print(sample_pos.index[0])
            # # print(sample_pos)
            
            # distance_list = distance_between_points(dat['points'].loc[sample_pos.index[0]],points_in_area_list)
            # return_dat['distance'] = distance_list

            # return_dat_sorted = return_dat.sort_values('distance')
            
            # return_dat_sorted['weight1'] = stats_distance1.pdf(return_dat_sorted['distance'])
            # return_dat_sorted['weight2'] = stats_distance2.pdf(return_dat_sorted['distance'])
            # return_dat_sorted['index'] = [names_mapping[i] for i in return_dat_sorted['Vitenskapelig navn']]
            
            

            # for k,kk,kkk in zip(list(return_dat_sorted['index']),list(return_dat_sorted['weight1']),list(return_dat_sorted['weight2'])):
            #     weight_tensor_normal250sd[j][k]=kk
            #     weight_tensor_normal500sd[j][k]=kkk
            
    
        # output_aug_kde05 = kde_augmentet_output(sample_pos_list,output, dat, 0.5)
        # output_aug_kde1 = kde_augmentet_output(sample_pos_list,output, dat, 1)
        # plants_in_area_tensor =torch.tensor(weight_tensor_1000)  

        # plants_in_area_tensor500 =torch.tensor(weight_tensor_500)  
        
        # plants_in_area_tensor_norm250sd = torch.tensor(weight_tensor_normal250sd)
        # plants_in_area_tensor_norm500sd = torch.tensor(weight_tensor_normal500sd)
        

        # output_aug = augment_output(output,plants_in_area_tensor)

        # output_aug500 = augment_output(output,plants_in_area_tensor500)

        
        
        # output_aug_norm250sd = augment_output(output,plants_in_area_tensor_norm250sd)
        

        # output_aug_norm500sd = augment_output(output,plants_in_area_tensor_norm500sd)
        
        
        
        loss = criterion(output, target)
        
        save_output_target_validate(model,output,target,epoch,i)
        
        # loss_aug_crit = nn.NLLLoss()

        # loss_aug = loss_aug_crit(output,target)
        # measure accuracy and record loss
        
        prec1, prec5, conf1 = accuracy(output.data, target, topk=(1, 5))
        
        # prec1_aug, prec5_aug,conf_aug1 = accuracy(output_aug.data, target, topk=(1, 5))
        # prec1_aug500, prec5_aug500,conf_500 = accuracy(output_aug500.data, target, topk=(1, 5))
        # prec1_aug_norm1, prec5_aug_norm1,conf_augnorm1 = accuracy(output_aug_norm250sd.data, target, topk=(1, 5))
        # prec1_aug_norm2, prec5_aug_norm2,conf_augnorm2 = accuracy(output_aug_norm500sd.data, target, topk=(1, 5))
        # prec1_aug_kde05, prec5_aug_kde05,conf_kde05 = accuracy(output_aug_kde05.data, target, topk=(1, 5))
        # prec1_aug_kde1, prec5_aug_kde1 , conf_kde1= accuracy(output_aug_kde1.data, target, topk=(1, 5))
        
        losses.update(loss.data.item(), input.size(0))
        
        top1.update(prec1.item(), input.size(0))
        top5.update(prec5.item(), input.size(0))

        # losses_aug.update(loss_aug.data.item(), input.size(0))
        # top1_aug.update(prec1_aug.item(), input.size(0))
        # top5_aug.update(prec5_aug.item(), input.size(0))
        # conf_aug.update(conf_aug1,1)
        
        # top1_aug500.update(prec1_aug500.item(), input.size(0))
        # top5_aug500.update(prec5_aug500.item(), input.size(0))
        # conf_aug500.update(conf_500,1)
        
        # top1_aug_norm1_250sd.update(prec1_aug_norm1.item(), input.size(0))
        # top5_aug_norm1_250sd.update(prec5_aug_norm1.item(), input.size(0))
        # conf_aug_norm1_250sd.update(conf_augnorm1,1)
        
        # top1_aug_norm2_500sd.update(prec1_aug_norm2.item(), input.size(0))
        # top5_aug_norm2_500sd.update(prec5_aug_norm2.item(), input.size(0))
        # conf_aug_norm2_500sd.update(conf_augnorm2,1)
        
        # top1_aug_kde05.update(prec1_aug_kde05.item(),input.size(0))
        # top5_aug_kde05.update(prec5_aug_kde05.item(),input.size(0))
        # conf_aug_kde05.update(conf_kde05,1)

        # top1_aug_kde1.update(prec1_aug_kde1.item(),input.size(0))
        # top5_aug_kde1.update(prec5_aug_kde1.item(),input.size(0))
        # conf_aug_kde1.update(conf_kde1,1)
        conf.update(conf1,1)
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 1 == 0:

            print('Test: [{0}/{1}]\n'.format(i, len(val_loader)))
            print('Time {batch_time.val:.3f} ({batch_time.avg:.3f})\n'.format(batch_time=batch_time))
            print('Loss {loss.val:.4f} ({loss.avg:.4f})\n'.format(loss=losses))
            print('Prec@1 {top1.val:.3f} ({top1.avg:.3f})\n'.format(top1=top1))
            print('Prec@5 {top5.val:.3f} ({top5.avg:.3f})\n'.format(top5=top5))
            # print(conf.sum.diag()/conf.sum.sum(1))
            # print('Loss_aug {loss_aug.val:.4f} {loss_aug.avg:.4f})\n'.format(loss_aug=losses_aug))
            # print('Prec@1_aug {top1_aug.val:.3f} ({top1_aug.avg:.3f})\n'.format(top1_aug=top1_aug))
            # print('Prec@5_aug {top5_aug.val:.3f} ({top5_aug.avg:.3f})\n'.format(top5_aug=top5_aug))
            # print(conf_aug.sum.diag()/conf_aug.sum.sum(1))
            # print('Prec@1_aug500 {top1_aug500.val:.3f} ({top1_aug500.avg:.3f})\n'.format(top1_aug500=top1_aug500))
            # print('Prec@5_aug500 {top5_aug500.val:.3f} ({top5_aug500.avg:.3f})\n'.format(top5_aug500=top5_aug500))
            # print(conf_aug500.sum.diag()/conf_aug500.sum.sum(1))
            # print('Prec@1_aug_norm1 {top1_aug_norm1_250sd.val:.3f} ({top1_aug_norm1_250sd.avg:.3f})\n'.format(top1_aug_norm1_250sd =top1_aug_norm1_250sd))
            # print('Prec@5_aug_norm1 {top5_aug_norm1.val:.3f} ({top5_aug_norm1.avg:.3f})\n'.format(top5_aug_norm1=top5_aug_norm1_250sd))
            # print(conf_aug_norm1_250sd.sum.diag()/conf_aug_norm1_250sd.sum.sum(1))
            # print('Prec@1_aug_norm2 {top1_aug_norm2_500sd.val:.3f} ({top1_aug_norm2_500sd.avg:.3f})\n'.format(top1_aug_norm2_500sd=top1_aug_norm2_500sd))
            # print('Prec@5_aug_norm2 {top5_aug_norm2.val:.3f} ({top5_aug_norm2.avg:.3f})\n'.format(top5_aug_norm2=top5_aug_norm2_500sd))
            # print(conf_aug_norm2_500sd.sum.diag()/conf_aug_norm2_500sd.sum.sum(1))
            # print('Prec@1_aug_kde05 {top1_aug_kde05.val:.3f} ({top1_aug_kde05.avg:.3f})\n'.format(top1_aug_kde05=top1_aug_kde05))
            # print('Prec@5_aug_kde05 {top5_aug_kde05.val:.3f} ({top5_aug_kde05.avg:.3f})\n'.format(top5_aug_kde05=top5_aug_kde05))
            # print(conf_aug_kde05.sum.diag()/conf_aug_kde05.sum.sum(1))
            # print('Prec@1_aug_kde1 {top1_aug_kde1.val:.3f} ({top1_aug_kde1.avg:.3f})\n'.format(top1_aug_kde1=top1_aug_kde1))
            # print('Prec@5_aug_kde1 {top5_aug_kde1.val:.3f} ({top5_aug_kde1.avg:.3f})\n'.format(top5_aug_kde1=top5_aug_kde1))
            # print(conf_aug_kde1.sum.diag()/conf_aug_kde1.sum.sum(1))
            # tt=conf_aug_kde1
        print(' * Prec@1 {top1.avg:.3f} Prec@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))
        


    return top1.avg

def augment_output(output, augment):
    output_aug = torch.nn.functional.softmax(output, dim=1)
    output_aug = output_aug * augment
    output_aug = output_aug * 1/torch.sum(output_aug) 
    return torch.log(output_aug)

def augment_output2(output, augment, zero_tensor_kde001):
    output_aug = torch.nn.functional.softmax(output, dim=1)
    augment = augment + zero_tensor_kde001*augment.min()
    output_aug = output_aug + augment
    output_aug = output_aug * 1/torch.sum(output_aug) 
    
    return torch.log(output_aug)

def kde_augmentet_output(sample_pos_list,output, dat, bandwidth):
    
    weight_tensor_kde = np.zeros((output.size(0),185))
    zero_tensor_kde = np.ones((output.size(0),185))
    for j, sample_pos in enumerate(sample_pos_list):
        within_square =get_points_within_square(sample_pos,dat)
        for plant_name in within_square.value_counts('Vitenskapelig navn').index:
            within_square_ = within_square[within_square['Vitenskapelig navn']==plant_name]
        
            pj = names_mapping[plant_name]
            pj_value = kernel_density_estimate_value(sample_pos,within_square_,bandwidth = bandwidth)
            weight_tensor_kde[j][pj] = pj_value
            zero_tensor_kde[j][pj] = 0
    weight_tensor_kde = torch.tensor(weight_tensor_kde)
    zero_tensor_kde = torch.tensor(zero_tensor_kde)
    
    output_aug_kde = augment_output2(output,weight_tensor_kde,zero_tensor_kde)

    return output_aug_kde
    
stats_distance1 = norm(
    loc=0, 
    scale=250
)

stats_distance2 = norm(
    loc=0, 
    scale=500
)
# def plants_in_area(,x)
#     more_than_x_obervations_of_target = False
#     while not more_than_x_obervations_of_target:
#         sample_plant_position()

In [11]:
start_epoch = 0

# args_resume = f'{model.name}_model_best.pth.tar'
args_resume = f'saved_models/{model.name}_checkpoint.pth.tar'
# if args_resume:
#     if os.path.isfile(args_resume):
#         print("=> loading checkpoint '{}'".format(args_resume))
#         checkpoint = torch.load(args_resume)
#         start_epoch = checkpoint['epoch']
#         best_prec1 = checkpoint['best_prec1']
#         model.load_state_dict(checkpoint['state_dict'])
#         optimizer.load_state_dict(checkpoint['optimizer'])
#         print("=> loaded checkpoint '{}' (epoch {})"
#               .format(args_resume, checkpoint['epoch']))
#         start_epoch = checkpoint['epoch']
#     else:
#         print("=> no checkpoint found at '{}'".format(args_resume))

# print('\n[INFO] Training Started')

@long_running
def train_model(model,LEARNING_RATE = LEARNING_RATE,NUM_EPOCHS = NUM_EPOCHS):
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                      momentum=0.9, weight_decay=1e-4, nesterov=True)
    train_loader = torch.utils.data.DataLoader(data_train, batch_size=64, shuffle=True, num_workers=2)
    val_loader = torch.utils.data.DataLoader(data_test, batch_size=64, shuffle=False, num_workers=2)
    
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                          momentum=0.9, weight_decay=1e-4, nesterov=True)
    criterion = nn.CrossEntropyLoss()
    
    args_resume = f'saved_models/{model.name}_checkpoint.pth.tar'
    if args_resume:
        if os.path.isfile(args_resume):
            print("=> loading checkpoint '{}'".format(args_resume))
            checkpoint = torch.load(args_resume)
            start_epoch = checkpoint['epoch']
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args_resume, checkpoint['epoch']))
            start_epoch = checkpoint['epoch']
        else:
            print("=> no checkpoint found at '{}'".format(args_resume))

            start_epoch = 0
            best_prec1 = 0
    print('\n[INFO] Training Started')

    for epoch in range(start_epoch, NUM_EPOCHS ):
        adjusted_rate = adjust_learning_rate(optimizer, epoch)
        
        if adjusted_rate:
            args_resume = f'saved_models/{model.name}_model_best.pth.tar'
            print("=> loading checkpoint '{}'".format(args_resume))
            checkpoint = torch.load(args_resume)
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args_resume, checkpoint['epoch']))

        # train for one epoch
        train(train_loader, model, criterion, optimizer, epoch)
        # evaluate on validation set
        prec1 = validate(val_loader, model, criterion,epoch,save_output=True)

        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
            'optimizer': optimizer.state_dict(),
        }, is_best)
        print('\n[INFO] Saved Model to leafsnap_model.pth')
        # torch.save(model, f'{model.name}_checkpoint.pth.tar')
        
train_model(model)

=> loading checkpoint 'saved_models/vgg19_pretrained_checkpoint.pth.tar'
=> loaded checkpoint 'saved_models/vgg19_pretrained_checkpoint.pth.tar' (epoch 45)

[INFO] Training Started

[Learning Rate] 0.001000
Epoch: [45][0/1475]	\Time 4.567 (4.567)	Data 4.229 (4.229)	Loss 0.0121 (0.0121)	Prec@1 98.438 (98.438)	Prec@5 100.000 (100.000)
Epoch: [45][100/1475]	\Time 0.123 (0.168)	Data 0.088 (0.128)	Loss 0.0199 (0.0141)	Prec@1 100.000 (99.134)	Prec@5 100.000 (100.000)


KeyboardInterrupt: 

In [ ]:
args_resume

In [ ]:
import warnings
warnings.filterwarnings("ignore")

args_resume = f'saved_models/{model.name}_checkpoint.pth.tar'
if args_resume:
    if os.path.isfile(args_resume):
        print("=> loading checkpoint '{}'".format(args_resume))
        checkpoint = torch.load(args_resume)
        start_epoch = checkpoint['epoch']
        best_prec1 = checkpoint['best_prec1']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}' (epoch {})"
              .format(args_resume, checkpoint['epoch']))
        start_epoch = checkpoint['epoch']
        NUM_EPOCHS = checkpoint['NUM_EPOCHS']
    else:
        print("=> no checkpoint found at '{}'".format(args_resume))

        start_epoch = 0
        best_prec1 = 0

validate(test_loader,model,criterion,epoch)

warnings.filterwarnings("default")

In [ ]:
# dat['points'][727362]


# dat.iloc[727362]
dat

In [16]:
import scipy
import numpy as np
import scipy.spatial as spatial
import matplotlib.pyplot as plt
import random 
import pandas as pd

from mpl_toolkits.basemap import Basemap
from sklearn.datasets import fetch_species_distributions
from sklearn.neighbors import KernelDensity
import copy
from pyproj import Proj, transform

def convert_utm_to_latlon(df, zone_number, zone_letter):
    utm_proj = Proj(proj='utm', zone=zone_number, ellps='WGS84', south=(zone_letter < 'N'))
    lonlat_proj = Proj(proj='latlong', datum='WGS84')
    lon, lat = transform(utm_proj, lonlat_proj, df['Østkoordinat'].values, df['Nordkoordinat'].values)
    
    return pd.DataFrame({'Longitude': lon, 'Latitude': lat})


dat = pd.read_csv('C:/Users/vjosv/master/dataset/top185_in_oslo_area.csv')
dat = dat[['Id','Østkoordinat','Nordkoordinat','Vitenskapelig navn']]




dat['points']= [[i,j] for i, j in zip(dat.Østkoordinat, dat.Nordkoordinat)]

df_latlon = convert_utm_to_latlon(dat, 33, 'N') 


dat['lat']=df_latlon['Latitude']
dat['long']=df_latlon['Longitude']



#oslo area:
#lowerleft, upperleft, upper right, lower right,


lat_long_oslo = [(58.998141, 9.574585), (60.351413, 9.574585), (60.351413, 12.540894),(58.998141,12.540894)]

# dat['Østkoordinat']>lat_long_oslo[0][0]


dat = dat.loc[(dat['lat']>lat_long_oslo[0][0])  & (dat['lat']<lat_long_oslo[1][0]) & (dat['long']>lat_long_oslo[0][1 ]) & (dat['long']<lat_long_oslo[2][1])]





points = dat[['Østkoordinat','Nordkoordinat']].to_numpy()




point_tree = spatial.cKDTree(points)


science_names = dat['Vitenskapelig navn'].unique()
indexes = [i for i in range(len(dat['Vitenskapelig navn'].unique()))]


random.seed(10)

random.shuffle(indexes)


names_mapping = {science_name : index for (science_name,index) in zip(science_names,indexes)}
index_mapping = {index : science_name for (science_name,index) in zip(science_names,indexes)}

def distance_between_points(point, list_of_points):
    return [np.sqrt(np.power(point[0]-lop[0],2)+np.power(point[1]-lop[1],2)) for lop in list_of_points]



def get_points_within(df_row, distance=1000):
    id = int(df_row['Id'].iloc[0])
    return_list = point_tree.query_ball_point([[int(df_row['Østkoordinat'].iloc[0]),int(df_row['Nordkoordinat'].iloc[0])]], distance)[0]
    return_dat = dat.iloc[return_list]
    return_list = list(return_dat['Id'])
    return_list.remove(id)
    return return_list
    
def sample_plant_position(plant,df):
    if type(plant) == str:
        return dat[dat['Vitenskapelig navn'] == plant].sample(1)
    elif type(plant) == int:
        return dat[dat['Vitenskapelig navn'] == index_mapping[plant]].sample(1)

def kernel_density_estimate_value(point_row,dat,bandwidth = 0.5):
    # print(point_row)
    # print(dat)
    if point_row.index[0] in list(dat.index):
        np_dat_lat_long = dat.drop(point_row.index[0])[['lat','long']].to_numpy()
    else:
        np_dat_lat_long = dat[['lat','long']].to_numpy()
    kde = KernelDensity(bandwidth=bandwidth)
    # print(np_dat_lat_long)
    if len(np_dat_lat_long)==0:
        return 0
    kde.fit(np_dat_lat_long)
    np_point = np.array([[point_row['lat'].iloc[0],point_row['long'].iloc[0]]])
    return np.exp(kde.score_samples(np_point))[0]

def get_points_within_square(point, dat,side_length = 3000):
    return_dat = dat[dat['Østkoordinat']>point['Østkoordinat'].iloc[0]-side_length]
    return_dat = return_dat[return_dat['Østkoordinat']<point['Østkoordinat'].iloc[0]+ side_length]
    return_dat = return_dat[return_dat['Nordkoordinat']>point['Nordkoordinat'].iloc[0]- side_length]
    return_dat = return_dat[return_dat['Nordkoordinat']<point['Nordkoordinat'].iloc[0]+ side_length]
    return return_dat

# within_square =get_points_within_square(v,dat)

# within_square







def get_points_within(df_row, distance=1000):
    id = int(df_row['Id'].iloc[0])
    return_list = point_tree.query_ball_point([[int(df_row['Østkoordinat'].iloc[0]),int(df_row['Nordkoordinat'].iloc[0])]], distance)[0]
    return_dat = dat.iloc[return_list]
    return_list = list(return_dat['Id'])
    return_list.remove(id)
    return return_list
    
def sample_plant_position(plant,df):
    if type(plant) == str:
        return dat[dat['Vitenskapelig navn'] == plant].sample(1)
    elif type(plant) == int:
        return dat[dat['Vitenskapelig navn'] == index_mapping[plant]].sample(1)




def get_knn_classifier(samples_pos_list,dat,n=1000):
    sample_pos_indexes = [sp.index[0] for sp in samples_pos_list if sp.index[0] in list(dat.index)]
    
    dat_removed_samples = dat.drop(index=sample_pos_indexes)
    category = []
    for k in dat_removed_samples['Vitenskapelig navn']:
        category.append(names_mapping[k])
        
    category = np.array(category)
    points_np= np.array(list(dat_removed_samples['points']))
    oversample = imblearn.over_sampling.KMeansSMOTE()
    warnings.filterwarnings("ignore")
    X, y = oversample.fit_resample(points_np, category) 
    warnings.filterwarnings("default")
    
    neigh = KNeighborsClassifier(n_neighbors=n)
    neigh.fit(X, y)
    return neigh

C:\Users\vjosv\AppData\Local\Temp\ipykernel_17404\3690604550.py:22: DtypeWarning: Columns (14,15,24,25,26,37,38,39,40,46) have mixed types. Specify dtype option on import or set low_memory=False.
  dat = pd.read_csv('C:/Users/vjosv/master/dataset/top185_in_oslo_area.csv')
C:\Users\vjosv\AppData\Local\Temp\ipykernel_17404\3690604550.py:17: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon, lat = transform(utm_proj, lonlat_proj, df['Østkoordinat'].values, df['Nordkoordinat'].values)


In [ ]:
np.array(dat['points'][0])
names_mapping['Convallaria majalis']

In [18]:

from geopy import distance




In [ ]:
samples_pos_list= []

for i in [0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1]:
    samples_pos_list.append(sample_plant_position(i,dat))
    



In [ ]:

knn = get_knn_classifier(samples_pos_list,dat)

In [ ]:
knn

In [ ]:
for i in aa:
    # print(i)
    if i ==3:
        print('ssssssssssssssss')

In [ ]:
a = list(sample_pos_list[-5]['points'])
a

In [ ]:
knn.predict_proba(a)

In [ ]:

category

points_np



In [ ]:
neigh = KNeighborsClassifier(n_neighbors=100)
neigh.fit(X, y)


In [ ]:

warnings.filterwarnings("ignore")
validate(test_loader,model,criterion)

warnings.filterwarnings("default")

In [ ]:
output = model(input_var)


loss = criterion(output, target_var)




In [ ]:
len(science_names)

len(dat['Vitenskapelig navn'].unique())

if 5 in range(5):
    print('s')

In [ ]:
checkpoint.state_dict()

In [ ]:
id = int(dat['Id'].iloc[0])

n = 10
df_row = dat
distance = 1000


v = point_tree.query_ball_point([[int(df_row['Østkoordinat'].iloc[n]),int(df_row['Nordkoordinat'].iloc[n])]], distance,return_sorted= True)[0]


return_dat = dat.iloc[v]
return_list = list(return_dat['points'])
def distance_between_points(point, list_of_points):
    return [np.sqrt(np.power(point[0]-lop[0],2)+np.power(point[1]-lop[1],2)) for lop in list_of_points]



distance_list = distance_between_points(dat['points'].iloc[n],return_list)

return_dat['distance'] = distance_list

return_dat_sorted = return_dat.sort_values('distance')
return_dat_sorted


indexes_in_area = [ names_mapping[i] for i in list(return_dat_sorted['Vitenskapelig navn'].unique())]

indexes_in_area

return_dat_sorted['Vitenskapelig navn'].unique()

return_dat_sorted


stats_distance0 = norm(
    loc=0, 
    scale=1000
)

stats_distance1 = norm(
    loc=0, 
    scale=500
)

stats_distance1.pdf(return_dat_sorted['distance'])

return_dat_sorted['weight'] = stats_distance1.pdf(return_dat_sorted['distance'])
return_dat_sorted['index'] = [names_mapping[i] for i in return_dat_sorted['Vitenskapelig navn']]


return_dat_sorted


In [ ]:
class_correct = list(0. for i in range(185))
class_total = list(0. for i in range(185))

# switch to evaluate mode
model.eval()
end = time.time()
for i, (input, target) in enumerate(val_loader):
    if USE_CUDA:
        input = input.cuda(non_blocking=True)
        target = target.cuda(non_blocking=True)
    input_var = torch.autograd.Variable(input, volatile=True)
    target_var = torch.autograd.Variable(target, volatile=True)

    # compute output
    output = model(input_var)
    
    break
    

In [ ]:
output
batch_size = target.size(0)

batch_size

_, pred = output.topk(5, 1, True, True)

predt = pred.t()
print(target.view(1, -1).expand_as(predt))
print(predt)

print(target)

print(predt.eq(target.view(1, -1).expand_as(predt)))


In [ ]:
# output.topk(5, 1, True, True)

pred.t()

In [ ]:
    maxk = max(topk)
    
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()

    correct = pred.eq(target.view(1, -1).expand_as(pred))
    
    res = []
    for k in topk:
    	correct_k = torch.sum(torch.reshape(correct[:k],(-1,) ).float()) #.double().sum(0)
    	res.append(correct_k.mul_(100.0 / batch_size))

In [ ]:

kernel_density_estimate_value(v,points_in_square_)

In [ ]:
# loss_aug = nn.NLLLoss(output,target)


loss_aug_crit = nn.NLLLoss()

loss_aug = loss_aug_crit(output,target)
loss_aug.data.item()

In [ ]:
print(output.topk(5, 1, True, True))

print(out.topk(5, 1, True, True))

In [ ]:
probs = torch.nn.functional.softmax(output, dim=1)
probs.topk(5, 1, True, True)

print(output)
print(torch.log(probs))
torch.nn.functional.log_softmax(output)

nn.NLLLoss(output)

In [ ]:
int(target_var)

In [ ]:

name_to_index ={}
for image in data_train.imgs:
    name_to_index[image[0].split('/')[3]]=image[1]
    
index_to_name = [i[0] for i in name_to_index.items()] 



In [ ]:
index_to_name

In [ ]:
# name_to_index

test_tensor = np.ones((output.size(0),185))*0.1

test_tensor

for i in test_tensor:
    for j in i:
        print(j)

In [ ]:
# !pip install pykrige
# !pip install skgstat

In [ ]:
import pykrige.kriging_tools as kt
from pykrige.ok import OrdinaryKriging
import matplotlib.pyplot as plt

In [ ]:
import scipy
import numpy as np
import scipy.spatial as spatial
import matplotlib.pyplot as plt
import random 
import pandas as pd
from mpl_toolkits.basemap import Basemap
from sklearn.datasets import fetch_species_distributions
from sklearn.neighbors import KernelDensity
import copy
from pyproj import Proj, transform

dat = pd.read_csv('dataset/lokasjon_top_185_with_height.csv')
dat = dat[['Id','Østkoordinat','Nordkoordinat','Vitenskapelig navn']]



dat['points'] = list(zip(dat.Østkoordinat, dat.Nordkoordinat))

points = dat[['Østkoordinat','Nordkoordinat']].to_numpy()

point_tree = spatial.cKDTree(points)


science_names = dat['Vitenskapelig navn'].unique()
indexes = [i for i in range(185)]
random.seed(10)

random.shuffle(indexes)


names_mapping = {science_name : index for (science_name,index) in zip(science_names,indexes)}
index_mapping = {index : science_name for (science_name,index) in zip(science_names,indexes)}


def get_points_within(df_row, distance=1000):
    id = int(df_row['Id'].iloc[0])
    return_list = point_tree.query_ball_point([[int(df_row['Østkoordinat'].iloc[0]),int(df_row['Nordkoordinat'].iloc[0])]], distance)[0]
    return_dat = dat.iloc[return_list]
    return_list = list(return_dat['Id'])
    return_list.remove(id)
    return return_list
    
def sample_plant_position(plant,df):
    if type(plant) == str:
        return dat[dat['Vitenskapelig navn'] == plant].sample(1)
    elif type(plant) == int:
        return dat[dat['Vitenskapelig navn'] == index_mapping[plant]].sample(1)


def convert_utm_to_latlon(df, zone_number, zone_letter):
    utm_proj = Proj(proj='utm', zone=zone_number, ellps='WGS84', south=(zone_letter < 'N'))
    lonlat_proj = Proj(proj='latlong', datum='WGS84')
    lon, lat = transform(utm_proj, lonlat_proj, df['Østkoordinat'].values, df['Nordkoordinat'].values)
    
    return pd.DataFrame({'Longitude': lon, 'Latitude': lat})



df_latlon = convert_utm_to_latlon(dat, 33, 'N') 


dat['lat']=df_latlon['Latitude']
dat['long']=df_latlon['Longitude']




In [ ]:
v = sample_plant_position(0,dat)
vv = get_points_within(v)
points_in_area = dat[dat['Id'].isin(vv)]


indexes_in_area = [ names_mapping[i] for i in list(points_in_area['Vitenskapelig navn'].unique())]


test_tensor = np.ones(185)*0.1
for i in indexes_in_area:
    test_tensor[i]=1



# out = output*tt
v

In [ ]:
def utm33_to_latlon(east,north):
    utm_proj = Proj(proj='utm', zone=33, ellps='WGS84', south=('N' < 'N'))
    lonlat_proj = Proj(proj='latlong', datum='WGS84')
    lon, lat = transform(utm_proj, lonlat_proj, east, north)
    return lat,lon

def get_basemap(df_row, side_length= 3000):
    lower_lat, lower_lon = utm33_to_latlon(df_row['Østkoordinat'].iloc[0]-side_length/2,df_row['Nordkoordinat'].iloc[0]-side_length/2)
    print(lower_lat,lower_lon)
    upper_lat, upper_lon = utm33_to_latlon(df_row['Østkoordinat'].iloc[0]+side_length/2,df_row['Nordkoordinat'].iloc[0]+side_length/2)
    
    map = Basemap(llcrnrlon=lower_lon,llcrnrlat=lower_lat,urcrnrlon=upper_lon,urcrnrlat=upper_lat,
                 resolution='i', lat_0 = 39.5, lon_0 = -3.25)


    print(lower_lon,upper_lon)
    xgrid = np.arange(lower_lon,upper_lon,0.00001)

    y0 = lower_lat
    y1 = upper_lat
    
    ygrid = np.arange(y0,y1,(y1-y0)/len(xgrid))
    return map, xgrid, ygrid
    
m, xgrid, ygrid = get_basemap(v)

points_in_square = get_points_within_square(v,dat)

latlong_np = points_in_square[['lat','long']].to_numpy()



fig, ax = plt.subplots(10, 1)
top_10_plants = list(points_in_square['Vitenskapelig navn'].value_counts()[:10].index)

fig.subplots_adjust(left=0.05, right=0.95, wspace=0.05)
# species_names = ['Bradypus Variegatus', 'Microryzomys Minutus']
cmaps = ['Purples', 'Reds']
for i, axi in enumerate(ax):
    axi.set_title(top_10_plants[i])
    mi= copy.deepcopy(m)
    
    points_in_square_ = points_in_square[points_in_square['Vitenskapelig navn']==top_10_plants[i]]

    
    ll = points_in_square_[['lat','long']].to_numpy()
    
    kde = KernelDensity(bandwidth=0.01)
    kde.fit(ll)
    
    mi.fillcontinents(color='white')

    mi.drawcoastlines()
    mi.drawcountries()
    # m.scatter(points_in_square['long'],points_in_square['lat'], zorder=3, cmap='rainbow', latlon=True);
    
    X, Y = np.meshgrid(xgrid[::5], ygrid[::5][::-1])
    xy = np.vstack([Y.ravel(), X.ravel()]).T
    
    Z = np.exp(kde.score_samples(xy))
    Z = Z.reshape(X.shape)
    Z[0]=1
    
    # plot contours of the density
    levels = np.linspace(0, Z.max(), 25)
    axi.contourf(X, Y, Z, levels=levels, cmap='Reds')
    vv = np.array([[v['lat'].iloc[0],v['long'].iloc[0]]])
              
    print(np.exp(kde.score_samples(vv)))


In [ ]:

v

vv = np.array([[v['lat'].iloc[0],v['long'].iloc[0]]])
              
np.exp(kde.score_samples(vv))


In [ ]:
xy

In [ ]:
within_square.groupby('Vitenskapelig navn').size()


In [ ]:
within_square.groupby('Vitenskapelig navn').size()

a =within_square[within_square['Vitenskapelig navn'] == 'Aegopodium podagraria']

H, xedges, yedges = np.histogram2d(a['Østkoordinat'], a['Nordkoordinat'], bins=10, range=None, density=None, weights=None)


fig = plt.figure(figsize=(7, 3))
ax = fig.add_subplot(131, title='imshow: square bins')
plt.imshow(H, interpolation='nearest', origin='lower',
        extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])




In [ ]:
within_square.groupby('Vitenskapelig navn').size()

a =within_square[within_square['Vitenskapelig navn'] == 'Acer platanoides']

H, xedges, yedges = np.histogram2d(a['Østkoordinat'], a['Nordkoordinat'], bins=10, range=None, density=None, weights=None)


fig = plt.figure(figsize=(7, 3))
ax = fig.add_subplot(131, title='imshow: square bins')
plt.imshow(H, interpolation='nearest', origin='lower',
        extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])



In [ ]:
within_square.groupby('Vitenskapelig navn').size()

a =within_square[within_square['Vitenskapelig navn'] == 'Vicia cracca']

H, xedges, yedges = np.histogram2d(a['Østkoordinat'], a['Nordkoordinat'], bins=10, range=None, density=None, weights=None)


fig = plt.figure(figsize=(7, 3))
ax = fig.add_subplot(131, title='imshow: square bins')
plt.imshow(H, interpolation='nearest', origin='lower',
        extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])





In [ ]:
# print(H[0])
# H
# print(x,y)
len(xedges)
xpoints = []
ypoints =[]
for i in range(len(xedges)-1):
    xpoints.append((xedges[i]+xedges[i+1])/2)
    ypoints.append((yedges[i]+yedges[i+1])/2)
xpoints
ypoints

In [ ]:

# H =H.T
# xedges
# yedges


x=[]
y=[]
yr=[]
phi = []
for i,ii in enumerate(H):
    for j,jj in enumerate(ii):
        x.append(xpoints[j])
        y.append(ypoints[i])
        yr.append(ypoints[i])
        if jj != 0:
            phi.append(jj*5)
        else:
            phi.append(0)
        

yr.reverse()


x = np.array(x)
y = np.array(y)
yr = np.array(yr)

phi = np.array(phi)

In [ ]:
sca = plt.scatter(x,y,c=phi)



print(x)
print(y)
print(phi)



OK = OrdinaryKriging(x,yr,phi, variogram_model='exponential',verbose = True, enable_plotting=True,)

In [ ]:
xx = np.arange(xedges[0],xedges[-1],10)
yy = np.arange(yedges[0],yedges[-1],10)
zstar, ss = OK.execute("grid",xx,yy)



In [ ]:
c = plt.imshow(zstar,extent=(xx[0],xx[-1],yy[0],yy[-1]))
# plt.scatter(x,y,c=phi,marker='.')


In [ ]:
import numpy as np
from scipy import stats
def measure(n):
    "Measurement model, return two coupled measurements."
    m1 = np.random.normal(size=n)
    m2 = np.random.normal(scale=0.5, size=n)
    return m1+m2, m1-m2
    
m1, m2 = measure(2000)
xmin = m1.min()
xmax = m1.max()
ymin = m2.min()
ymax = m2.max()
X, Y = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
positions = np.vstack([X.ravel(), Y.ravel()])
values = np.vstack([m1, m2])
kernel = stats.gaussian_kde(values)
Z = np.reshape(kernel(positions).T, X.shape)

import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.imshow(np.rot90(Z), cmap=plt.cm.gist_earth_r,
          extent=[xmin, xmax, ymin, ymax])
ax.plot(m1, m2, 'k.', markersize=2)
ax.set_xlim([xmin, xmax])
ax.set_ylim([ymin, ymax])
plt.show()

In [ ]:

X, Y = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
X.ravel()
values
kernel
positions = np.vstack([X.ravel(), Y.ravel()])

# plt.scatter(positions[0],positions[1],c='k')

In [ ]:
x = np.array([11,22,55,33,77,88,11,22])
y = np.array([69,90,1,34,33,88,48,22])
phi = np.array([66,99,11,33,33,88,44,22])

OK = OrdinaryKriging(x,y,phi, variogram_model='exponential',verbose = True, enable_plotting=True,)
xx = np.arange(x.min(),x.max(),0.1)
yy = np.arange(y.min(),y.max(),0.1)

zstar, ss = OK.execute("grid",xx,yy)

plt.imshow(zstar,extent=(xx[0],xx[-1],yy[0],yy[-1]))
plt.scatter(x,np.flip(y),c='k',marker='.')

In [ ]:
plt.imshow(ss,extent=(xx[0],xx[-1],yy[0],yy[-1]))

y
plt.scatter(np.flip(x),np.flip(y),c='k',marker='.')



In [ ]:

# dat.iloc[vv]
points_in_area = dat[dat['Id'].isin(vv)]


indexes_in_area = [ names_mapping[i] for i in list(points_in_area['Vitenskapelig navn'].unique())]


In [ ]:
indexes_in_area


stats_distance = norm(
    loc=0, 
    scale=100
)

points_in_area


In [ ]:
np.ones((185,10))

In [ ]:
print(output.topk(5, 1, True, True))

print(out.topk(5, 1, True, True))

In [ ]:
b = np.ones((2,185))

a = torch.tensor(b)
a.size()

In [ ]:
from sklearn.datasets import fetch_species_distributions

data = fetch_species_distributions()

# Get matrices/arrays of species IDs and locations
latlon = np.vstack([data.train['dd lat'],
                    data.train['dd long']]).T
species = np.array([d.decode('ascii').startswith('micro')
                    for d in data.train['species']], dtype='int')

In [ ]:
# !pip install --upgrade matplotlib
from mpl_toolkits.basemap import Basemap
from sklearn.datasets import fetch_species_distributions

def construct_grids(batch):
    """Construct the map grid from the batch object

    Parameters
    ----------
    batch : Batch object
        The object returned by :func:`fetch_species_distributions`

    Returns
    -------
    (xgrid, ygrid) : 1-D arrays
        The grid corresponding to the values in batch.coverages
    """
    # x,y coordinates for corner cells
    xmin = batch.x_left_lower_corner + batch.grid_size
    xmax = xmin + (batch.Nx * batch.grid_size)
    ymin = batch.y_left_lower_corner + batch.grid_size
    ymax = ymin + (batch.Ny * batch.grid_size)

    # x coordinates of the grid cells
    xgrid = np.arange(xmin, xmax, batch.grid_size)
    # y coordinates of the grid cells
    ygrid = np.arange(ymin, ymax, batch.grid_size)

    return (xgrid, ygrid)


xgrid, ygrid = construct_grids(data)

# plot coastlines with basemap
m = Basemap(projection='cyl', resolution='c',
            llcrnrlat=ygrid.min(), urcrnrlat=ygrid.max(),
            llcrnrlon=xgrid.min(), urcrnrlon=xgrid.max())
m.drawmapboundary(fill_color='#DDEEFF')
m.fillcontinents(color='#FFEEDD')
m.drawcoastlines(color='gray', zorder=2)
m.drawcountries(color='gray', zorder=2)

# plot locations
m.scatter(latlon[:, 1], latlon[:, 0], zorder=3,
          c=species, cmap='rainbow', latlon=True);

In [ ]:
# !pip install sklearn
from sklearn.neighbors import KernelDensity
# Set up the data grid for the contour plot
X, Y = np.meshgrid(xgrid[::5], ygrid[::5][::-1])
land_reference = data.coverages[6][::5, ::5]
land_mask = (land_reference > -9999).ravel()
xy = np.vstack([Y.ravel(), X.ravel()]).T
xy = np.radians(xy[land_mask])

# Create two side-by-side plots
fig, ax = plt.subplots(1, 2)
fig.subplots_adjust(left=0.05, right=0.95, wspace=0.05)
species_names = ['Bradypus Variegatus', 'Microryzomys Minutus']
cmaps = ['Purples', 'Reds']

for i, axi in enumerate(ax):
    axi.set_title(species_names[i])
    
    # plot coastlines with basemap
    m = Basemap(projection='cyl', llcrnrlat=Y.min(),
                urcrnrlat=0, llcrnrlon=200,
                urcrnrlon=100, resolution='c', ax=axi)
                # urcrnrlat=Y.max(), llcrnrlon=X.min(),
                # urcrnrlon=X.max(), resolution='c', ax=axi)
    m.drawmapboundary(fill_color='#DDEEFF')
    m.drawcoastlines()
    m.drawcountries()
    
    # construct a spherical kernel density estimate of the distribution
    kde = KernelDensity(bandwidth=0.03, metric='haversine')
    kde.fit(np.radians(latlon[species == i]))

    # evaluate only on the land: -9999 indicates ocean
    Z = np.full(land_mask.shape[0], -9999.0)
    Z[land_mask] = np.exp(kde.score_samples(xy))
    Z = Z.reshape(X.shape)

    # plot contours of the density
    levels = np.linspace(0, Z.max(), 25)
    axi.contourf(X, Y, Z, levels=levels, cmap=cmaps[i])

In [ ]:
latlon[:, 0]

In [ ]:
map = Basemap(projection='ortho',lat_0=45,lon_0=-100,resolution='l')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# set up orthographic map projection with
# perspective of satellite looking down at 50N, 100W.
# use low resolution coastlines.
map = Basemap(projection='cyl',lat_0=0,lon_0=0,resolution='l')
# draw coastlines, country boundaries, fill continents.
map.drawcoastlines(linewidth=0.25)
map.drawcountries(linewidth=0.25)
map.fillcontinents(color='coral',lake_color='aqua')
# draw the edge of the map projection region (the projection limb)
map.drawmapboundary(fill_color='aqua')
# draw lat/lon grid lines every 30 degrees.
# map.drawmeridians(np.arange(0,360,30))
# map.drawparallels(np.arange(-90,90,30))
# make up some data on a regular lat/lon grid.
nlats = 73; nlons = 145; delta = 2.*np.pi/(nlons-1)
lats = (0.5*np.pi-delta*np.indices((nlats,nlons))[0,:,:])
lons = (delta*np.indices((nlats,nlons))[1,:,:])
wave = 0.75*(np.sin(2.*lats)**8*np.cos(4.*lons))
mean = 0.5*np.cos(2.*lats)*((np.sin(2.*lats))**2 + 2.)
# compute native map projection coordinates of lat/lon grid.
x, y = map(lons*180./np.pi, lats*180./np.pi)
# contour data over the map.
cs = map.contour(x,y,wave+mean,15,linewidths=1.5)
plt.title('contour lines over filled continent background')
plt.show()

In [ ]:
dat['Vitenskapelig navn'].value_counts()


In [ ]:
d = dat[dat['Vitenskapelig navn']=='Stellaria nemorum']#'Lupinus polyphyllus']


ll = d[['lat','long']].to_numpy()

In [ ]:
np.arange

In [ ]:
xgrid = np.arange(3,34,0.1)
ygrid = np.arange(57,72,(72-57)/len(xgrid))


X, Y = np.meshgrid(xgrid[::5], ygrid[::5][::-1])
xy = np.vstack([Y.ravel(), X.ravel()]).T
# xy = np.radians(xy[land_mask])



In [ ]:


map = Basemap(llcrnrlon=3.,llcrnrlat=57.,urcrnrlon=34.,urcrnrlat=72.,
             resolution='i', lat_0 = 39.5, lon_0 = -3.25)
# map.drawmapboundary(fill_color='blue')
# map.drawcountries(linewidth=0.25)
map.fillcontinents(color='white')#,lake_color='blue')
# map.scatter(d['long'],d['lat'], zorder=3, cmap='rainbow', latlon=True);

map.drawmapboundary(fill_color='#DDEEFF')
map.drawcoastlines()
map.drawcountries()

kde = KernelDensity(bandwidth=0.03, metric='haversine')
kde.fit(ll)

# evaluate only on the land: -9999 indicates ocean
# Z = np.full(land_mask.shape[0], -9999.0)
Z = np.exp(kde.score_samples(xy))
Z = Z.reshape(X.shape)

Z=Z*100
# plot contours of the density
levels = np.linspace(0, Z.max(), 25)

axi.contourf(X, Y, Z, levels=levels, cmap='Reds')

plt.show()





In [ ]:
fig, ax = plt.subplots(1, 1)
fig.subplots_adjust(left=0.05, right=0.95, wspace=0.05)
species_names = ['Bradypus Variegatus', 'Microryzomys Minutus']
cmaps = ['Purples', 'Reds']
ax=[ax]
for i, axi in enumerate(ax):
    axi.set_title(species_names[i])
    
    # plot coastlines with basemap
    # m = Basemap(projection='cyl', llcrnrlat=Y.min(),
    #             urcrnrlat=Y.max(), llcrnrlon=X.min(),
    #             urcrnrlon=X.max(), resolution='c', ax=axi)
    m = Basemap(llcrnrlon=3.,llcrnrlat=57.,urcrnrlon=34.,urcrnrlat=72.,
             resolution='i', lat_0 = 39.5, lon_0 = -3.25)
    m.drawmapboundary(fill_color='#DDEEFF')
    m.drawcoastlines()
    m.drawcountries()
    m.scatter(d['long'],d['lat'], zorder=3, cmap='rainbow', latlon=True);
    # construct a spherical kernel density estimate of the distribution
    kde = KernelDensity(bandwidth=0.5)
    kde.fit(ll)

    # evaluate only on the land: -9999 indicates ocean
    # Z = np.full(land_mask.shape[0], -9999.0)
    Z = np.exp(kde.score_samples(xy))
    Z = Z.reshape(X.shape)
 
    # plot contours of the density
    levels = np.linspace(0, Z.max(), 25)
    axi.contourf(X, Y, Z, levels=levels, cmap=cmaps[i])

In [ ]:
# Set up the data grid for the contour plot
xgrid, ygrid = construct_grids(data)

xgrid = np.arange(3,34,0.1)
ygrid = np.arange(57,72,(72-57)/len(xgrid))


X, Y = np.meshgrid(xgrid[::5], ygrid[::5][::-1])
land_reference = data.coverages[6][::5, ::5]

xy = np.vstack([Y.ravel(), X.ravel()]).T


fig, ax = plt.subplots(1, 1)
ax = [ax] 
fig.subplots_adjust(left=0.05, right=0.95, wspace=0.05)
species_names = ['Bradypus Variegatus', 'Microryzomys Minutus']
cmaps = ['Purples', 'Reds']
for i, axi in enumerate(ax):
    axi.set_title(species_names[i])
    
    # plot coastlines with basemap
    # m = Basemap(projection='cyl', llcrnrlat=Y.min(),
    #             urcrnrlat=Y.max(), llcrnrlon=X.min(),
    #             urcrnrlon=X.max(), resolution='c', ax=axi)

    map = Basemap(llcrnrlon=3.,llcrnrlat=57.,urcrnrlon=34.,urcrnrlat=72.,
             resolution='i', lat_0 = 39.5, lon_0 = -3.25)
    map.drawmapboundary(fill_color='#DDEEFF')
    map.drawcoastlines()
    map.drawcountries()
    
    # construct a spherical kernel density estimate of the distribution
    kde = KernelDensity(bandwidth=1)
    kde.fit(ll)
    
    # evaluate only on the land: -9999 indicates ocean
    

    Z= np.exp(kde.score_samples(xy))

    Z = Z.reshape(X.shape)

    
    # map.scatter(d['long'],d['lat'], zorder=3, latlon=True);
    
    # plot contours of the density
    levels = np.linspace(0, Z.max(), 15)
    axi.contourf(X, Y, Z, levels=levels, cmap='Reds')

In [ ]:
Z

In [ ]:
kde.score_samples(xy)

In [ ]:
ll

In [ ]:
within_square 
sample_pos 
output_aug_kde001 
weight_tensor_kde001 

In [ ]:
within_square

In [ ]:
len(within_square['Vitenskapelig navn'].unique())

In [ ]:


torch.exp(output_aug_kde001[0])

In [ ]:

def kernel_density_estimate_value(point_row,dat,bandwidth = 1):
    # print(point_row)
    # print(dat)
    if point_row.index[0] in list(dat.index):
        np_dat_lat_long = dat.drop(point_row.index[0])[['lat','long']].to_numpy()
    else:
        np_dat_lat_long = dat[['lat','long']].to_numpy()
    kde = KernelDensity(bandwidth=bandwidth)
    # print(np_dat_lat_long)
    if len(np_dat_lat_long)==0:
        return 1
    kde.fit(np_dat_lat_long)
    np_point = np.array([[point_row['lat'].iloc[0],point_row['long'].iloc[0]]])
    return np.exp(kde.score_samples(np_point))[0]

# kernel_density_estimate_value()
within_square_ = within_square[within_square['Vitenskapelig navn']== 'Hypericum maculatum']

kernel_density_estimate_value(sample_pos,within_square_)

In [ ]:
weight_tensor_kde001

In [ ]:
np_dat_lat_long = within_square_[['lat','long']].to_numpy()
kde = KernelDensity(bandwidth=1)
# print(np_dat_lat_long)

kde.fit(np_dat_lat_long)


In [ ]:
kde.score(np.array([[sample_pos['lat'].iloc[0],sample_pos['long'].iloc[0]]]))

In [ ]:
weight_tensor_kde001

In [ ]:
weight_tensor_kde001.min()



In [ ]:
zero_tensor_kde001

In [ ]:
zero_tensor_kde001.sum()+159

In [ ]:
# !pip install geopandas
import urllib
import geojson
import geopandas as gpd
from shapely.geometry import Point, Polygon
import json

# kilde https://data.artsdatabanken.no/Natur_i_Norge/Landskap/Typeinndeling
# data_geo = gpd.read_file('dataset/landskapsgradient.geojson')


# f = open('dataset/landskapsgradient.geojson')
# a =json.load(f)

In [ ]:

# p = Point(89100,6973100)

# # p2.within(poly)
# p.within(data['geometry'][0])
# data
print(a.keys())

# a['features'][300]
p = Polygon(a['features'][0]['geometry']['coordinates'])
p

In [ ]:
from owslib.wms import WebMapService
# S&version=1.1.1&
wms = WebMapService('https://wms.artsdatabanken.no/?map=/maps/mapfiles/la.map&?&request=GetCapabilities&service=WM', version='1.1.1')

from pyproj import Proj, transform
import json




In [ ]:
a = wms.contents['LA']

In [ ]:
wms.getfeatureinfo

In [ ]:
 img = wms.getfeatureinfo(    
      layers=['LA'], 
      styles=['default'], 
      srs='EPSG:32633', 
      bbox=(245953+1,6654310+1,245953-1,6654310-1), 
      size=(600, 300), 
      format='application/json', 
        xy=(0,0),
      )

In [ ]:
img.read()

In [ ]:
dat = pd.read_csv('dataset/lokasjon_top_185_with_height_flora.csv')
dat.head()

In [ ]:

outProj = Proj(init='epsg:32633') 

inProj = Proj(init='epsg:32633') 
region_size=(100,100)

p = (245953,6654310)


loc = p

x, y = transform(inProj,outProj,loc[0],loc[1])

xupper = int(round(x - region_size[0] / 2))
xlower = int(round(x + region_size[0] / 2))
yupper = int(round(y - region_size[1] / 2))
ylower = int(round(y + region_size[1] / 2))
bbox = (xupper, yupper, xlower, ylower)
#bbox=(-11375363,3757392,-11372226,3759102)
size=region_size
size=(1333, 716)

wms_version = '1.1.1'


In [ ]:
bbox

In [ ]:
info = wms.getmap(
    layers=['1_NATURAL_COL0R'],
    srs='EPSG:32633',
    bbox = bbox,
    size = size,
    info_format='application/json',
    xy = (0, 0)
)
# info = json.loads(info.read())
info.read()

In [ ]:
from tqdm import tqdm
# flora_list = []
# kode_list = []
# index_list = []
ii = 446376
for i,p in enumerate(tqdm(dat['points'][ii:])):
    point = Point(p[0],p[1])
    for g, flora, kode in zip(data_geo['geometry'],data_geo['navn'],data_geo['kode']):
        if point.within(g):
            flora_list.append(flora)
            kode_list.append(kode)
            index_list.append(i+ii)
            break 


In [ ]:
# print(len(kode_list))
# print(i)
# index_list[-1]+


import pandas as pd

b =pd.read_csv('nature_type_geo.csv')
b

In [ ]:
# df = pd.DataFrame([])
df['kode']= kode_list
df['index'] = index_list
df['flora'] = flora_list

In [ ]:
# df.to_csv('nature_type_geo.csv')

In [ ]:
# kode_list = kode_list[:417604-874]
# index_list = index_list[:417604-874]
# flora_list = flora_list[:417604-874]

In [ ]:
index_list[-1]

In [ ]:
dat

In [ ]:
def f(x):
    s = [1,2]
    return s.append(x)

In [ ]:
a,b,c= f(2)

In [ ]:


dat = pd.read_csv('dataset/lokasjon_top_185_with_height.csv')
dat2 = pd.read_csv('dataset/nature_type_geo.csv')

In [ ]:
len(dat['Unnamed: 0'])
len(dat2['index'])

In [ ]:
dat2=dat2[['flora','kode','index']]


In [ ]:
dat3 =pd.merge(
    dat,
    dat2,
    how="left",
    left_on ='Unnamed: 0',
    right_on = 'index'
    )

In [ ]:
len(dat3['Id'])

In [ ]:
len(dat['Id'])

In [ ]:
dat

In [ ]:
grouped = dat[['Vitenskapelig navn','height']].groupby('Vitenskapelig navn')

grouped.mean()

In [ ]:
v =grouped.mean()

In [ ]:
v.loc['Acer platanoides'][0]

In [ ]:
dat2

In [ ]:
flora_ = list(dat2['flora'])
kode_ = list(dat2['kode'])
index_ = list(dat2['index'])

kode = ['U'] * len(list(dat['Id']))
flora = ['unknown'] * len(list(dat['Id']))

# index = list(dat[''])

In [ ]:
for i,j in enumerate(index_):
    flora[j] = flora_[i]
    kode[j] = kode_[i]

In [ ]:
dat['flora'] = flora
dat['kode'] = kode 

In [ ]:
dat

In [ ]:
df = pd.from_c